In [3]:
import re
import pdfplumber
import csv
from difflib import SequenceMatcher
import os


In [4]:

def extract_left_right_columns(pdf_path):
    """
    PDF에서 신구조문대비표 영역의 좌측(현행) / 우측(개정) 열을 나누어 추출
    """
    left_clauses, right_clauses = [], []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            words = page.extract_words()
            if not words:
                continue
            midpoint = (page.bbox[2] - page.bbox[0]) / 2
            left_lines = {}
            right_lines = {}
            for w in words:
                y0 = round(w["top"])
                text = w["text"].strip()
                if not text:
                    continue
                target = left_lines if w["x0"] < midpoint else right_lines
                if y0 not in target:
                    target[y0] = []
                target[y0].append(text)
            for y in sorted(left_lines):
                left_clauses.append(" ".join(left_lines[y]))
            for y in sorted(right_lines):
                right_clauses.append(" ".join(right_lines[y]))
    return left_clauses, right_clauses


def merge_by_clause(lines):
    """제XX조, ①, 1. 기준으로 조문 묶기"""
    clause_blocks = []
    current_clause = ""
    clause_id = ""
    for line in lines:
        if re.match(r"제\d+조", line):
            if current_clause:
                clause_blocks.append((clause_id, current_clause.strip()))
            clause_id = re.findall(r"제\d+조(?:\s*제\d+항)?", line)[0]
            current_clause = line
        elif re.match(r"[\(\[]?\d+[\)\.]", line) or re.match(r"[\u2460-\u2473]", line):
            current_clause += "\n" + line
        else:
            current_clause += " " + line
    if current_clause:
        clause_blocks.append((clause_id, current_clause.strip()))
    return clause_blocks


def compare_clauses(old_clauses, new_clauses, threshold=0.85):
    """
    조문 ID 기준으로 비교 후 변경 유형 분류
    """
    results = []
    old_dict = {cid: text for cid, text in old_clauses}
    new_ids = set(cid for cid, _ in new_clauses)

    for cid, new_text in new_clauses:
        old_text = old_dict.get(cid)
        if old_text:
            sim = SequenceMatcher(None, old_text, new_text).ratio()
            if sim < threshold:
                results.append((cid, old_text, new_text, "변경"))
        else:
            results.append((cid, "", new_text, "신설"))

    for cid, old_text in old_clauses:
        if cid not in new_ids:
            results.append((cid, old_text, "", "삭제"))

    return results


def save_to_csv(data, path, law_name=""):
    with open(path, mode="w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["법률명", "조문ID", "현행", "개정", "변경유형"])
        for cid, old, new, chg in data:
            writer.writerow([law_name, cid, old, new, chg])


In [ ]:
# === 실행 ===

folder_path = r"C:\Jimin\cg_DeltaLaw\data\no_upload"
file_list = os.listdir(folder_path)

for file in file_list:
    path = os.path.join(folder_path, file)
    left, right = extract_left_right_columns(path)
    old_clauses = merge_by_clause(left)
    new_clauses = merge_by_clause(right)
    results = compare_clauses(old_clauses, new_clauses)
    save_to_csv(results, "조문_비교결과.csv", law_name=file)


In [9]:
import os
import csv

base_path = r"C:\Jimin\cg_DeltaLaw\data"
folder_path = os.path.join(base_path, "no_upload")
output_path = os.path.join(base_path, "processed", "조문_전체비교결과.csv")

all_results = []

for file in os.listdir(folder_path):
    if not file.endswith(".pdf"):
        continue
    file_path = os.path.join(folder_path, file)

    # 법률명 추출 예시: 파일명에서 추출하거나 PDF 첫 페이지에서 파싱
    law_name = os.path.splitext(file)[0]  # 단순히 파일명 기반
    try:
        left, right = extract_left_right_columns(file_path)
        old_clauses = merge_by_clause(left)
        new_clauses = merge_by_clause(right)
        results = compare_clauses(old_clauses, new_clauses)

        # 각 결과에 법률명 추가
        for cid, old, new, chg in results:
            all_results.append([law_name, cid, old, new, chg])
    except Exception as e:
        print(f"❌ 오류 발생: {file} → {e}")

In [10]:
# 마지막에 CSV로 저장
with open(output_path, mode="w", encoding="utf-8-sig", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["법률명", "조문ID", "현행", "개정", "변경유형"])
    writer.writerows(all_results)

In [11]:
import pandas as pd

file_path = r"C:\Jimin\cg_DeltaLaw\data\processed\조문_전체비교결과.csv"
df = pd.read_csv(file_path)

df.head()

df.columns

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   법률명     284 non-null    object
 1   조문ID    266 non-null    object
 2   현행      268 non-null    object
 3   개정      74 non-null     object
 4   변경유형    284 non-null    object
dtypes: object(5)
memory usage: 11.2+ KB


In [12]:
import pandas as pd

file_path = r"C:\Jimin\cg_DeltaLaw\data\processed\조문_전체비교결과.csv"
df = pd.read_csv(file_path)

df.to_excel(r"C:\Jimin\cg_DeltaLaw\data\processed\조문_전체비교결과.xlsx", index=False)

## ❌ 1차 파이프라인 실패 원인 정리

### 1. 조문 ID 매칭 실패
- `"제X조"` 형태 추출이 불안정하거나 일부 누락됨
- 현행 ↔ 개정 조문 간 1:1 매칭 실패

### 2. 좌/우 열 추출 실패
- 일부 PDF에서 신구조문대비표의 **좌우 열 인식이 어긋남**
- `midpoint` 기준 단순 분할로는 **모든 포맷 커버 불가**

### 3. 불필요한 줄 포함
- `"현행과 같음"`, `"생략"`, `"------"` 등 **의미 없는 텍스트 포함**
- 전처리 부족으로 결과 정합성 저하

### 4. 변경유형 분류 정확도 낮음
- `SequenceMatcher` 기반 유사도 판정이 **너무 단순**
- 실제 변경 아님에도 `"변경"`으로 오분류하거나 반대 상황 발생

### 5. 법률명 식별 미흡
- 단순히 파일명으로만 법률명을 추정
- **정확한 법률명 추출 로직 부재**

---

📌 위 문제점을 기준으로 **v2 파이프라인 설계에 반영 예정**